In [2]:
# !pip install optuna

In [3]:
import optuna
from optuna.visualization.matplotlib import plot_param_importances
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras.backend import clear_session
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.layers import Conv1D, MaxPool1D, Dense, Flatten, Input
from keras.models import Sequential
from sklearn.model_selection import train_test_split

/home/pedro/miniconda3/envs/op297/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-06 13:53:15.406167: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
CLASSES = 10
EPOCHS = 15

Função auxiliar para obtenção e separação dos dados em `train`, `test` e `valid`

In [5]:
def get_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    img_x, img_y = x_test.shape[1], x_test.shape[2]
    x_train = x_train.reshape(-1, img_x, img_y, 1).astype("float32") / 255
    x_test = x_test.reshape(-1, img_x, img_y, 1).astype("float32") / 255

    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                      test_size=0.05)

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)

Funções auxiliares para definição do espaço de busca paramétrico:


*   `get_config`: hiperparametrocos base, comum a todos os modelos
*   `get_cnn_config`: parâmetros específicos para modelo convolucional
*   `get_mlp_config`: parâmetros específicos para modelo totalmente conectado

Utiliza-se o objeto `trial`, que é instanciado e obtido após a chamada do `study.optimize`. O `trial` fornece funções como `suggest_float`, `suggest_categorical`, `suggest_int`, etc. Estas funções devem ser utilizadas de acordo com a característica do parâmetro a ser amostrado, e podem receber um range min/max do valores do parâmetro ou uma lista de categorias.



In [6]:
'''  
Funções auxiliares para definição do espaço de busca paramétrico:  
  * `get_config`: hiperparametrocos base, comum a todos os modelos  
  * `get_mlp_config`: parâmetros específicos para modelo totalmente conectado  
Utiliza-se o objeto `trial`, que é instanciado e obtido após a chamada do `study.optimize`. O `trial` fornece funções como `suggest_float`, `suggest_categorical`, `suggest_int`, etc. Estas funções devem ser utilizadas de acordo com a característica do parâmetro a ser amostrado, e podem receber um range min/max do valores do parâmetro ou uma lista de categorias.  
''' 

def get_config(trial):
  config = {
      "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-1),
      "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64, 128, 256]),
      "optimizer": trial.suggest_categorical("optimizer", [0, 1, 2]),
      "activation": trial.suggest_categorical("activation",
       ["relu", "tanh", "sigmoid"]),
      "activation_out": trial.suggest_categorical("activation_out",
       ["softmax", "sigmoid"]),
  }

  return config


def get_cnn_config(trial):
  config = get_config(trial)
  config["n_hidden"] = trial.suggest_int("n_hidden", 1, 2)
  config["kernel_1"] = trial.suggest_categorical("kernel_1", [3, 5])
  config["kernel_2"] = trial.suggest_categorical("kernel_2", [3, 5])
  config["filters"] = trial.suggest_categorical("filters", [32, 64])
  config["stride"] = trial.suggest_categorical("stride", [1, 2])
  config["pool_size"] = trial.suggest_categorical("pool_size", [3, 5])

  return config


def get_mlp_config(trial):
  config = get_config(trial)
  config["n_layers"] = trial.suggest_int("n_layers", 1, 3)
  for l in range(config["n_layers"]):
    config[f"n_neurons_{l}"] = trial.suggest_int(f"n_neurons_{l}", 16, 1024)

  return config


def get_optimizer(config):
    possible_optims = [
        tf.keras.optimizers.SGD(learning_rate=config["learning_rate"]),
        tf.keras.optimizers.Adam(learning_rate=config["learning_rate"]),
        tf.keras.optimizers.RMSprop(learning_rate=config["learning_rate"]),
    ]

    return possible_optims[config["optimizer"]]


Funções auxiliares para definição de dois modelos base:

*   Convolucional (`get_cnn`)
*   Totalmente conectado (`get_mlp`)



In [7]:
def get_cnn(input_shape, config):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv1D(
            filters=config["filters"],
            kernel_size=config["kernel_1"],
            activation=config["activation"])
    )
    model.add(Conv1D(
        filters=config["filters"],
        kernel_size=config["kernel_2"],
        activation=config["activation"],)
    )
    model.add(MaxPool1D(pool_size=config["pool_size"]))
    model.add(Flatten())
    model.add(Dense(CLASSES, activation="softmax"))

    return model

In [8]:
def get_mlp(config):
  model = Sequential()
  model.add(Flatten())
  print(config.keys())
  for l in range(config["n_layers"]):
    model.add(Dense(config[f"n_neurons_{l}"], activation=config["activation"]))
  model.add(Dense(CLASSES, activation=config["activation_out"]))

  return model


### Definição da função objetivo
1. Obter os dados;
2. Definir configurações de espaço de busca paramêtrico;
3. Definir o modelo;
4. Treinar e avaliar;

In [9]:
def objective(trial):
    clear_session()

    (x_train, y_train), (x_val, y_val), (x_test, y_test) = get_mnist()

    # input_shape = (x_train.shape[1], x_train.shape[2], 1)
    # configs = get_cnn_config(trial)
    # model = get_cnn(input_shape, configs)

    configs = get_mlp_config(trial)
    model = get_mlp(configs)

    optimizer = get_optimizer(configs)

    model.compile(
        loss="sparse_categorical_crossentropy",
        # loss="mse",
        optimizer=optimizer,
        metrics=["accuracy"],
    )

    monitor = EarlyStopping(
        monitor="val_loss",
        min_delta=1e-4,
        patience=10,
        verbose=0,
        mode="auto",
        restore_best_weights=True,
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_val, y_val),
        shuffle=True,
        batch_size=configs["batch_size"],
        epochs=EPOCHS,
        verbose=True,
    )

    return model.evaluate(x_test, y_test, verbose=0)[1]



# Criação

1.   Criar um [`optuna.Study`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study), utilizando a função [`create_study`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.create_study.html).

  *   Um estudo corresponde a uma tarefa de otimização, i.e., um conjunto de tentativas.


2.   Definir uma função objetivo a ser otimizada, através da chamada do método [`optimize`](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study.optimize) do objeto `optuna.Study`
   *   Definir os principais parametros como: função a ser otimizada (`objective`), número de tentativas (`n_trials`), número de jobs paralelos (`n_jobs`), etc.

   *   A função objetivo será executava `n_trials` vezes, até a conclusão da otimização como foi definida.

3. Plotar o gráfico de importância dos parâmetros através do [`plot_param_importances`](https://optuna.readthedocs.io/en/stable/reference/visualization/generated/optuna.visualization.plot_param_importances.html) do pacote `optuna.visualization`

4. Tratar o dataframe final e salvar os resultados.



In [10]:
# 1) Criar um estudo (tarefa de otimização / conjunto de tentativas)
study = optuna.create_study(direction="maximize")

# 2) Definir uma função objetivo a ser otimizada
study.optimize(objective, n_trials=100, n_jobs=-1, show_progress_bar=True)

# 3) Plotar o gráfico de importância dos parâmetros
plot_param_importances(study)

date = pd.Timestamp.now().strftime("%d%m%Y_%H:%M:%S")
plt.savefig(f"importances_{date}.png")

print(f"Numero de otimizacoes realizadas: {len(study.trials)}")

print("Melhor resultado:")
trial = study.best_trial

print(f"  Acc: {trial.value}")

print("  Params: ")
optims = ["SGD", "Adam", "RMSprop"]
for key, value in trial.params.items():
    if key != "optimizer":
      print(f"    {key}: {value}")
    else:
      print(f"    {key}: {optims[value]}")

# 4) Tratar o DataFrame final e salvar os resultados
res_df = study.trials_dataframe()
columns = [c for c in res_df.keys() if c.startswith("params_")] + ["value"]
res_df["params_optimizer"] = res_df["params_optimizer"].apply(lambda x: optims[x])
res_df.columns = [c.replace("params_", "") for c in res_df.columns]
csv_path = f"optuna_{date}.csv"
res_df.to_csv(csv_path, index=False)

[I 2024-04-06 13:53:16,517] A new study created in memory with name: no-name-c2ccd80c-4727-4deb-836a-8ba839b834b6
  0%|          | 0/100 [00:00<?, ?it/s]

2024-04-06 13:53:18.696557: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-06 13:53:18.710261: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-06 13:53:18.710632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0', 'n_neurons_1', 'n_neurons_2'])dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0'])
dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0', 'n_neurons_1'])

dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0', 'n_neurons_1'])
dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0', 'n_neurons_1'])
dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0', 'n_neurons_1', 'n_neurons_2'])
dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_layers', 'n_neurons_0'])
dict_keys(['learning_rate', 'batch_size', 'optimizer', 'activation', 'activation_out', 'n_

2024-04-06 13:53:43.135033: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-04-06 13:53:43.168447: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f96e0231390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-06 13:53:43.168475: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2024-04-06 13:53:43.439954: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-04-06 13:53:43.522533: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-04-06 13:53:44.503030: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` 

 11/446 [..............................] - ETA: 21s - loss: 0.1445 - accuracy: 0.9517Epoch 13/15
Epoch 4/15
703/891 [======================>.......] - ETA: 11s - loss: 13.4518 - accuracy: 0.1003

...

153/891 [====>.........................] - ETA: 42s - loss: 0.0666 - accuracy: 0.979561

Best trial: 31. Best value: 0.9784:   8%|▊         | 8/100 [04:16<11:06,  7.25s/it]

 109/1782 [>.............................] - ETA: 1:40 - loss: 0.5360 - accuracy: 0.8578

Best trial: 31. Best value: 0.9784:   9%|▉         | 9/100 [04:16<08:12,  5.41s/it]

181/891 [=====>........................] - ETA: 43s - loss: 0.1101 - accuracy: 0.9697Epoch 1/15
Epoch 1/15
 120/3563 [>.............................] - ETA: 3:45 - loss: 4.1994 - accuracy: 0.0964